In [2]:
# creazione tabella frequenza
import numpy as np
import pywt
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from scipy.fft import rfft
from scipy.fft import fft

from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
import pandas as pd
from imblearn.over_sampling import SMOTE



from sklearn.metrics import classification_report
df = pd.read_csv(r"C:\Users\umber\Desktop\MANUTENZIONE\dataset_train_finale.csv")
df = pd.concat([df]*2, ignore_index=True)

x_test=pd.read_csv(r"C:\Users\umber\Desktop\MANUTENZIONE\dataset_test_finale.csv")
df = df[["Case","Condition","P1","P2","P3","P4","P5","P6","P7"]]
#ALLENAMENTO MODELLO
test = x_test[["P1","P2","P3","P4","P5","P6","P7"]]
X=df[["P1","P2","P3","P4","P5","P6","P7"]]

df_train = pd.DataFrame()
peaks_train = {}


for colonna in X.columns:
    df_train[colonna+"_FFT"] = rfft(X[colonna].values)



num_rows = len(df_train)
case_values = []
current_case = 1
for i in range(num_rows):
    case_values.append(current_case)
    if (i + 1) % 1201 == 0:
        current_case += 1


# Aggiungi la colonna "case" al dataframe
df_train['case'] = case_values



last_col_index = len(df_train.columns) - 1

for colonna in df_train.columns[:last_col_index]:
    peaks_train[colonna+"peaks"] = find_peaks(df_train[colonna], threshold= 1, distance=10) #7 elementi con le posizioni

for colonna in df_train.columns[:last_col_index]:
    df_train[colonna] = df_train[colonna][peaks_train[colonna+"peaks"][0]]

df_train = df_train.fillna(0)
df_train = df_train.applymap(lambda x: np.real(x))
def replace_case(value):
    if value <= 105:
        return 0
    else:
        return 1

#df_train['case'] = df_train['case'].apply(replace_case)
df_train = df_train[(df_train.iloc[:, :7] != 0).any(axis=1)]

X = df_train [["P1_FFT","P2_FFT","P3_FFT","P4_FFT","P5_FFT","P6_FFT","P7_FFT"]]
y = df_train["case"]

smote = SMOTE()
X_smote, y_smote = smote.fit_resample(X, y)

df_smote = pd.concat([pd.DataFrame(X_smote, columns=X.columns), pd.DataFrame(y_smote, columns=["case"])], axis=1)
df_smote



c:\Users\umber\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\series.py:917: ComplexWarning: Casting complex values to real discards the imaginary part
  arr = np.asarray(values, dtype=dtype)


,P1_FFT,P2_FFT,P3_FFT,P4_FFT,P5_FFT,P6_FFT,P7_FFT,case
0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,46.300093,1
1,0.000000e+00,105.484182,0.000000,216.405343,0.000000,0.000000,0.000000,1
2,0.000000e+00,0.000000,0.000000,0.000000,0.000000,110.181413,0.000000,1
3,3.909590e+01,0.000000,89.230939,0.000000,40.291068,0.000000,0.000000,1
4,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,154.154740,1
...,...,...,...,...,...,...,...,...
81238,0.000000e+00,0.000000,0.000000,0.000000,0.000000,5.820209,0.000000,177
81239,6.825922e+00,0.000000,133.986687,0.000000,0.000000,0.000000,0.000000,177
81240,6.656880e-13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,177
81241,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,63.072069,177


In [3]:
import pandas as pd

# Supponiamo che il tuo DataFrame si chiami df e abbia una colonna chiamata 'case'

# Creiamo un DataFrame vuoto
new_df = pd.DataFrame()

# Per ogni valore univoco nella colonna 'case'
for value in df_smote['case'].unique():
    # Estraiamo solo le righe con quel valore nella colonna 'case'
    subset = df_smote[df_smote['case'] == value]
    
    # Calcoliamo quante righe mancano per arrivare a 1201
    missing_rows = 1201 - len(subset)
    
    # Se mancano righe, aggiungiamo righe con zeri
    if missing_rows > 0:
        # Creiamo un DataFrame temporaneo con righe di zeri
        temp_df = pd.DataFrame([[value] + [0] * (len(df_smote.columns) - 1)] * missing_rows, columns=df_smote.columns)
        
        # Aggiungiamo il DataFrame temporaneo al nuovo DataFrame
        new_df = pd.concat([new_df, subset, temp_df], ignore_index=True)
    else:
        # Se il numero di righe è già uguale a 1201, aggiungiamo semplicemente il subset al nuovo DataFrame
        new_df = pd.concat([new_df, subset], ignore_index=True)

new_df

,P1_FFT,P2_FFT,P3_FFT,P4_FFT,P5_FFT,P6_FFT,P7_FFT,case
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,46.300093,1
1,0.000000,105.484182,0.000000,216.405343,0.000000,0.000000,0.000000,1
2,0.000000,0.000000,0.000000,0.000000,0.000000,110.181413,0.000000,1
3,39.095898,0.000000,89.230939,0.000000,40.291068,0.000000,0.000000,1
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,154.154740,1
...,...,...,...,...,...,...,...,...
212572,177.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
212573,177.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
212574,177.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
212575,177.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0


In [4]:
#creazione tabella tempo
df = pd.read_csv(r"C:\Users\umber\Desktop\MANUTENZIONE\dataset_train_finale.csv")
df_train1 = df[["P1","P2","P3","P4","P5","P6","P7","Case"]]
def replace_case(value):
    if value <= 105:
        return 0
    else:
        return 1

#df_train1['Case'] = df_train1['Case'].apply(replace_case)
df_train1.rename(columns={'Case': 'case'}, inplace=True)
df_train1




C:\Users\umber\AppData\Local\Temp\ipykernel_44596\3296136663.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train1.rename(columns={'Case': 'case'}, inplace=True)


,P1,P2,P3,P4,P5,P6,P7,case
0,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1
1,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1
2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1
3,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1
4,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1
...,...,...,...,...,...,...,...,...
212572,1.948734,1.912482,1.904362,1.835654,1.924899,1.857220,1.920020,177
212573,1.948823,1.899824,1.884360,1.825497,1.927513,1.846068,1.919081,177
212574,1.957784,1.911383,1.893740,1.859805,1.940953,1.861668,1.950562,177
212575,1.970451,1.950009,1.945417,1.913911,1.953648,1.835381,1.983321,177


In [4]:
new_df = new_df[["P1_FFT","P2_FFT","P3_FFT","P4_FFT","P5_FFT","P6_FFT","P7_FFT"]]
df_train1 = df_train1.join(new_df)
df_train1


,P1,P2,P3,P4,P5,P6,P7,case,P1_FFT,P2_FFT,P3_FFT,P4_FFT,P5_FFT,P6_FFT,P7_FFT
0,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1,0.000000,52.742091,0.000000,108.202671,0.0,0.000000,0.00000
1,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1,0.000000,0.000000,0.000000,0.000000,0.0,55.090706,0.00000
2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1,19.547949,0.000000,0.000000,0.000000,0.0,0.000000,0.00000
3,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,77.07737
4,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1,0.000000,0.000000,80.416843,0.000000,0.0,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212572,1.948734,1.912482,1.904362,1.835654,1.924899,1.857220,1.920020,177,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212573,1.948823,1.899824,1.884360,1.825497,1.927513,1.846068,1.919081,177,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212574,1.957784,1.911383,1.893740,1.859805,1.940953,1.861668,1.950562,177,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212575,1.970451,1.950009,1.945417,1.913911,1.953648,1.835381,1.983321,177,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
percorso_file = r"C:\Users\umber\Desktop\aiutoo.csv"

# Salvataggio del DataFrame nel file CSV specificato
df_train1.to_csv(percorso_file, index=False)

In [5]:
X_smote = df_train1[["P1","P2","P3","P4","P5","P6","P7","P1_FFT","P2_FFT","P3_FFT","P4_FFT","P5_FFT","P6_FFT","P7_FFT"]]
y_smote = df_train1["case"]

x_train_sv, x_test_sv, y_train_sv, y_test_sv = train_test_split(X_smote, y_smote, test_size=0.20)
model = RandomForestClassifier(max_depth = None, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 200, random_state=42)
model.fit(x_train_sv, y_train_sv)
predictions = model.predict(x_test_sv)

print(classification_report(y_test_sv, predictions))

MemoryError: could not allocate 371195904 bytes